# Arquitectura, modelado y gestión de datos en Data Science
<hr />

## Semana 3 - Ejercicio práctico 3 - Grupo 10
### Realizado por:
💻 Mayra Cecilia Salazar Grandes   
💻 José Manuel Espinoza Bone

# 0️⃣ Inicializar/Preparar ambiente

Aquí instalamos las dependencias externas que no se encuentran en la biblioteca estándar de Python para lograr que el presente notebook se puede ejecutar sin problemas.

In [ ]:
import sys
import subprocess
def RunCommand(commandList: list[str]):
    print("    ⏳ Ejecutando: ", " ".join(commandList))
    result = subprocess.run(commandList, stdout=subprocess.DEVNULL,stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        print(result.stderr) 

print("🟦 Instalando las dependencias externas")
RunCommand([sys.executable, "-m", "pip", "install", "numpy"]) 
RunCommand([sys.executable, "-m", "pip", "install", "pandas"])
RunCommand([sys.executable, "-m", "pip", "install", "tabulate"])
RunCommand([sys.executable, "-m", "pip", "install", "matplotlib"])
RunCommand([sys.executable, "-m", "pip", "install", "seaborn"])
RunCommand([sys.executable, "-m", "pip", "install", "requests"])
RunCommand([sys.executable, "-m", "pip", "install", "openpyxl"]) 

In [ ]:
#Importando las dependencias
import pandas 
import pandas as pd
import numpy 
import numpy as np
import matplotlib.pyplot as pyplot
import seaborn
import requests
from tabulate import tabulate
import datetime
from pathlib import Path

# Mostrar todas las filas
pandas.set_option('display.max_rows', None)
# Mostrar todas las columnas
pandas.set_option('display.max_columns', None)
# Ajustar el ancho máximo de columna (para no truncar texto)
pandas.set_option('display.max_colwidth', None)

# --- Funciones utilitarias ---
# Función para mostrar la información del DataFrame
def ShowTableInfo(df:pandas.DataFrame, title):
    display(f"ℹ️ {title} ℹ️".upper())
    df.info()
    display()

# Función para mostrar las n primeras filas del DataFrame.
def ShowTableHead(df:pandas.DataFrame, title:str, headQty=10):
    display(f"ℹ️ {title}: Primeros {headQty} elementos.".upper())
    display(df.head(headQty))
    display()

# Función para mostrar las n últimas filas del DataFrame.
def ShowTableTail(df:pandas.DataFrame, tailQty=10):
    display(f"ℹ️ Últimos {tailQty} elementos.".upper())
    display(df.tail(tailQty))
    display()

# Función para mostrar la estadística descriptiva de todas las columnas del DataFrame, por tipo de dato.
def ShowTableStats(df: pandas.DataFrame):
    display("ℹ️ Estadística descriptiva".upper())
    numeric_types = ['int64', 'float64', 'Int64', 'Float64']
    numeric_cols = df.select_dtypes(include=numeric_types)
    if not numeric_cols.empty:
        display("    🔢 Columnas numéricas")
        numeric_desc = numeric_cols.describe().round(2)
        display(numeric_desc)
        #print(tabulate(numeric_desc, headers='keys', tablefmt='fancy_grid')) 
    non_numeric_types = ['object', 'string', 'bool', 'category']
    non_numeric_cols = df.select_dtypes(include=non_numeric_types)
    if not non_numeric_cols.empty:
        display("    🔡 Columnas no numéricas")
        non_numeric_desc = non_numeric_cols.describe()
        display(non_numeric_desc)
        #print(tabulate(non_numeric_desc, headers='keys', tablefmt='fancy_grid'))
    datetime_cols = df.select_dtypes(include=['datetime'])
    if not datetime_cols.empty:
        display("    📅 Columnas fechas")
        datetime_desc = datetime_cols.describe()
        display(datetime_desc)
        #print(tabulate(datetime_desc, headers='keys', tablefmt='fancy_grid'))

# Función para mostrar los valores nulos o NaN de cada columna en un DataFrame
def ShowNanValues(df: pandas.DataFrame):
    display(f"ℹ️ Contador de valores Nulos".upper())
    nulls_count = df.isnull().sum()
    nulls_df = nulls_count.reset_index()
    nulls_df.columns = ['Columna', 'Cantidad_Nulos']
    display(nulls_df)
    display()

# 1️⃣ Dataset Oil Spill. FASE 1

✅ Importar dataset y realizar análisis exploratorio.

In [ ]:
oil_spill_uri = "https://raw.githubusercontent.com/UIDE-Tareas/1-Arquitectura-Modelado-Gestion-Datos-Data-Science-Tarea3/refs/heads/main/Datasets/oil-spill.csv"
display(f"🟦 Importando dataset desde \"{oil_spill_uri}\"")
try:
    df_original= pd.read_csv (oil_spill_uri,header=None)
    display(f"Dataset cargado")
except:
    display("Ocurrió un error al leer el dataset!.")
    sys.exit()

In [ ]:
df_Main= df_original.copy()
ShowTableInfo(df_Main, "Análisis Exploratorio".upper())
ShowTableStats(df_Main)
ShowTableHead(df_Main, "Dataset original")
ShowNanValues(df_Main)

✅Identifica cuántas y cuales columnas tienen valores únicos.    
✅Eliminalas e imprime el tamaño del dataset antes y después.    

In [ ]:
#df_Main = df_original.copy()
# Tamaño original dataset
df_eliminados_unique = df_Main.copy()
print(f"Tamaño original del dataset: {df_Main.shape}")
# Identificar columnas con un solo valor único
columnas_valor_unico = [col for col in df_Main.columns if df_Main[col].nunique() == 1]
print(f"Número de columnas con un solo valor único: {len(columnas_valor_unico)}")
print(columnas_valor_unico)
# Eliminar esas columnas
df_eliminados_unique.drop(columns=columnas_valor_unico, inplace=True)
print(f"Tamaño después de eliminar columnas con un solo valor único : {df_eliminados_unique.shape}")
ShowTableInfo(df_eliminados_unique, "Luego de eliminar uniques")

✅ Porcentaje de valores únicos por columna con respecto al total                                            
✅ Define límite de incidencia(umbral), identificar columnas por debajo de ese umbral y eliminarlas mostrando su porcentaje

**Límite de incidencia**

Al analizar los porcentajes de valores únicos por columna podemos evidenciar lo siguiente:

* Los porcentajes menores al 1% podrían contener poca información útil, producto a sus valores muy constantes. 

Para esta práctica se decidio que el umbral para eliminar los columnas sería los porcentajes que están menores al 1%. La variable que maneja este valor es UMBRAL_MIN.

In [ ]:

## Función para mostrar el porcentaje de valores únicos con respecto al total de filas.
def  ShowUniquePercentVsTotal(df: pandas.DataFrame):
    display ("🟦 Porcentaje de valores únicos por columna".upper())
    for col in df.columns:
        total = df[col].shape[0]
        unicos = df[col].nunique()
        porcentaje = (unicos / total) * 100
        # Impresión valores únicos por columna 
        print(f"{col}: {unicos} valores únicos → {porcentaje:.2f}% del total {total}")
    display()

# Función para eliminar las columnas en base al umbral, si drop es True se elimina directamente del dataframe original, si output es True se imprime el valor de la columna a eliminar.
def eliminar_columnas_incidencia_porcentaje(df:pandas.DataFrame, umbral, drop: bool, output: bool = False):
    umbral = abs(umbral)
    if output:
        display(f"🟦 Columnas a ser eliminadas cuyo porcentaje de uniques es < {umbral}%")
    # Creando un diccionario donde se guardará la columna y porcentaje a ser eliminado
    columnas_eliminadas = pandas.DataFrame({"Name": [], "PorcentajeUniques": [], "TotalUniques": [], "Total": []})
    df_columnas_eliminadas = pandas.DataFrame()
    for col in df.columns:
        total = df[col].shape[0]
        unicos = df[col].nunique()
        porcentaje = (unicos / total) * 100
        if (porcentaje < umbral):
            # Impresión de columnas a eliminar
            if output:
                print(f"{col}: {unicos} valores únicos → {porcentaje:.2f}% del total")
            df_columnas_eliminadas[col] = df[col].copy()
            columnas_eliminadas.loc[len(columnas_eliminadas)] = [f"{col}", porcentaje, unicos, total]
             # Eliminar columna que cumple la condición
            if drop:
                df.drop(columns=col, inplace= True)
    return df, df_columnas_eliminadas,columnas_eliminadas

df_eliminado_incidencias = df_Main.copy()
ShowUniquePercentVsTotal(df_eliminado_incidencias)

# Umbrales
UMBRAL_MIN = 1
df_eliminado_incidencias,_, columnas_eliminadas = eliminar_columnas_incidencia_porcentaje(df_eliminado_incidencias, UMBRAL_MIN, True, False)

ShowTableInfo(df_eliminado_incidencias, "Eliminado incidencias")
ShowTableHead(df_eliminado_incidencias, "Eliminado incidencias")

✅ Analizar y graficar cuantas columnas serán eliminadas usando varios umbrales. La variable UMBRALES tiene diferentes umbrales para ser probados.

In [ ]:
UMBRALES = [0.1,0.5,1,5,10,25,50,75,90,95,100, 101, 500, -1000,-75]
cantidad_eliminadas_por_umbral = pandas.DataFrame({"Umbral": [], "Cantidad": []})
for umbral in UMBRALES:
    df_original = df_Main.copy()
    df_original,df_eliminadas, columnas_eliminadas = eliminar_columnas_incidencia_porcentaje(df_original, umbral, False, False)
    # Agregando nuevo elemento al DF con el valor del umbral y la cantidad de columnas eliminadas
    cantidad_eliminadas_por_umbral.loc[len(cantidad_eliminadas_por_umbral)] = [umbral, columnas_eliminadas.shape[0]]

display("🟦 Cantidad de Columnas a ser eliminadas por umbral(cantidad valores únicos que están debajo)".upper())
display(cantidad_eliminadas_por_umbral)

import matplotlib.pyplot as plt
import seaborn as sns
# Función para mostrar el gráfico de barras de cada umbral versus el número de columnas eliminadas para ese umbral
def graficar_umbrales(df: pandas.DataFrame):
    plt.figure(figsize=(12, 6))
    sns.barplot(x="Umbral", y="Cantidad", data=df, palette="magma", hue="Umbral")
    plt.xticks(rotation=90)
    plt.xlabel("Umbral")
    plt.ylabel("Cantidad de columnas a ser eliminadas")
    plt.title("Cantidad de Columnas a ser eliminadas por umbral(cantidad valores únicos que están debajo)")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

graficar_umbrales(cantidad_eliminadas_por_umbral)





✅ Visualiza la distribución de los valores de las columnas filtradas, mediante gráficos Barplot, histograma o similar. Para este notebook se escogió el umbral con menor número de columnas eliminadas que es 5%, en el cual se eliminan 13 columnas; todo esto con el objetivo de generar la menor cantidad de gráficos.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
UMBRAL_MIN = 0.5


def DibujarBarras(df: pandas.DataFrame, umbral):
    display(f"🟦 Valores de las columnas eliminadas para el umbral {umbral}".upper())
    df_original = df.copy()
    df_original, df_eliminadas, columnas_eliminadas = eliminar_columnas_incidencia_porcentaje(df_original, umbral, False, False)
    
    for col in df_eliminadas.columns:
        plt.figure(figsize=(10, 5))
        sns.histplot(df_eliminadas[col].values, bins=df_eliminadas[col].nunique(), kde=False, color="skyblue", edgecolor="black")
        plt.xlabel(f"Valor: {col}")
        plt.ylabel("Frecuencia")
        plt.title(f"Frecuencias de característica {col}")
        plt.tight_layout()
        plt.show()

UMBRALES = [5]
for umbral in UMBRALES:
    DibujarBarras(df_Main, umbral)
    



✅ Realiza un análisis de varianza por columna y comenta los resultados

In [ ]:
import matplotlib.pyplot as plt
# Calcular varianza de cada columna numérica
varianzas = df_Main.var(numeric_only=True).sort_values(ascending=False).reset_index()
varianzas.columns = [ "Columna", "Varianza"]
# Mostrar las varianzas ordenadas de mayor a menor
print("Características por varianza ordenada de mayor a menor\n")
print(varianzas)
# Crear el gráfico

plt.figure(figsize=(12, 6))
sns.barplot(x="Columna", y="Varianza", data=varianzas, palette="magma", hue="Columna")
plt.xticks(rotation=90)
plt.xlabel("Umbral")
plt.ylabel("Cantidad de columnas a ser eliminadas")
plt.title("Cantidad de Columnas a ser eliminadas por umbral(cantidad valores únicos que están debajo)")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

Analisis varianza
* Al analizar el gráfico se puede observar que la mayoría de las columnas tienen una varianza muy baja, casi cercana a cero, posiblemente contienen valores casi constantes, lo que significa que aportan poca o ninguna información al modelo.
* La columna "Característica_5" tiene una varianza extremadamente alta, del orden de 1e13, lo cual la hace una variable dominante en términos de escala y dispersión, por tal motivo se debería revisar si no esta en una escala diferente al resto, probablemente necesite normalización o estandarización para evitar que domine al entrenar un modelo de machine learning.



# 2️⃣ Dataset  Pima-indians-diabetes. Fase 2.